In [ ]:
import numpy as np
from tensorboardX import SummaryWriter
from matplotlib import pyplot as plt
import tensorflow as tf
import random
import os

In [ ]:
import numpy as np
from tensorboardX import SummaryWriter
from matplotlib import pyplot as plt
import tensorflow as tf
import random
import os
from Adjoint_regularizition import Regularized
from Adjoint_network import TwoNets
from Classical_correction import ClassicalCorrection
import Operators.Load_PAT2D_data as PATdata
import platform
from Framework import approx_PAT_matrix as ApproxPAT
from Framework import exact_PAT_operator as ExactPAT

if platform.node() == 'motel':
    prefix = '/local/scratch/public/sl767/ModelCorrection/'
else:
    prefix = ''

matrix_path = prefix+'Data/Matrices/threshSingleMatrix4Py.mat'
# data_path = prefix+'Data/vessels/'
# saves_path = prefix+'Saves/vessels/'
data_path = prefix+'Data/balls64/'
saves_path = prefix+'Saves/balls64/'
vessel_saves_path = prefix+'Saves/vessels/'

print(saves_path)
print(data_path)

train_append = 'trainDataSet.mat'
test_append = 'testDataSet.mat'
data_sets = PATdata.read_data_sets(data_path + train_append, data_path + test_append, vessels=False, flip90=False)
# data_sets = PATdata.read_data_sets(data_path + train_append, data_path + test_append)

vessel_path = prefix+'Data/vessels/'
vessel_train_append = 'trainDataSet_VesselClean.mat'
vessel_test_append = 'testDataSet_VesselClean.mat'
vessel_data = PATdata.read_data_sets(vessel_path + vessel_train_append, vessel_path + vessel_test_append, vessels=False, flip90=False)

INPUT_DIM = (64,64)
OUTPUT_DIM = (64,64)

approx = ApproxPAT(matrix_path=matrix_path, input_dim=INPUT_DIM, output_dim=OUTPUT_DIM)
exact = ExactPAT(matrix_path=matrix_path, input_dim=INPUT_DIM, output_dim=OUTPUT_DIM)

# Experiments to compare generalizability between data sets

In [ ]:
image = vessel_data.test.default_batch(32)

In [ ]:
noise_level = 0.01
n_steps = 300
TV = 7e-4
n_print = 10
verbose = True

In [ ]:
regularized = TwoNets(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets, noise_level = noise_level,
                         experiment_name='TwoNetsRekursiveOldNet', savepoint=None)
res_TN = regularized.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
regularized.end()

In [ ]:
n_steps = 10000
classical = ClassicalCorrection(path=saves_path, true_np=exact, appr_np=approx, lam=.001, data_sets=data_sets,
                         noise_level = noise_level, experiment_name='ClassicalRegularization', characteristic_scale=.25)
res_AEM = classical.log_optimization(image, recursions=n_steps, step_size=0.5, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
classical.end()

##### Trained on vessels, evaluated on balls

In [ ]:
TV = 7e-4
noise_level = 0.01
n_steps = 300
n_print = 10
verbose = True

In [ ]:
regularized = TwoNets(path=vessel_saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=vessel_data, noise_level = noise_level,
                         experiment_name='TwoNetsRekursiveAug', savepoint=None, characteristic_scale=.25)


In [ ]:
classical = ClassicalCorrection(path=vessel_saves_path, true_np=exact, appr_np=approx, lam=.001, data_sets=vessel_data,
                         noise_level = noise_level, experiment_name='ClassicalRegularization', characteristic_scale=.25)

In [ ]:
image = data_sets.test.default_batch(32)

In [ ]:
n_steps = 2000
TV = 7e-3
n_print = 10
verbose = True
res = {}

In [ ]:
TV = 1e-2
res[f'FB_{TV}'] = regularized.log_optimization(image, recursions=n_steps, step_size=0.1, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)

In [ ]:
### This is the original parameter used
TV = 7e-4
res[f'FB_{TV}']  = regularized.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)

In [ ]:
### This is the original parameter used
TV = 7e-4
res['Uncorrected']  = regularized.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print, operator='Approx')

In [ ]:
TV = 7e-5
res[f'AEM_{TV}']  = classical.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)

In [ ]:
n_steps = 20000
TV = 7e-4
res[f'AEM_{TV}']  = classical.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)

In [ ]:
TV = 7e-3
res[f'AEM_{TV}']  = classical.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)

In [ ]:
TV = 7e-2
res[f'AEM_{TV}']  = classical.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)

### Performances
- AEM: 0.58
- FA, old TV Para: 0.40
- FA, new TV Para: 0.35

- Accurate: 0.11
- Uncorrected: 0.55
- AEM, trained on balls: 0.31
- FA, trained on balls: 0.15

In [ ]:
fig_path = '/local/scratch/public/sl767/ModelCorrection/Figures/CrossData/'

In [ ]:
fig_name = 'TrainedVessels_EvalBalls_comp.png'
vmax = image[0,...,0].max()
fig = plt.figure(figsize=(12,3))
plt.subplot(141)
im = plt.imshow(image[0,...,0]/vmax, vmin=0, vmax=1)
plt.axis('off')
plt.title('Ground Truth')
plt.subplot(142)
plt.imshow(res['Uncorrected'][1][0,...,0]/vmax, vmin=0, vmax=1)
plt.axis('off')
plt.title('Uncorrected')
plt.subplot(143)
plt.imshow(res['FB_0.0007'][1][0,...,0]/vmax, vmin=0, vmax=1)
plt.axis('off')
plt.title('For.-Adj. Original TV param.')
plt.subplot(144)
plt.imshow(res['FB_0.01'][1][0,...,0]/vmax, vmin=0, vmax=1)
plt.axis('off')
plt.title('For.-Adj. New TV param.')
cbar_ax = fig.add_axes([0.93, 0.15, 0.015, 0.7])
fig.colorbar(im, cax=cbar_ax)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
fig_name = 'TrainedVessels_EvalBalls_renormalized_comp.png'
vmax = image[0,...,0].max()
fig = plt.figure(figsize=(12,3))
plt.subplot(141)
im = plt.imshow(image[0,...,0])
plt.axis('off')
plt.title('Ground Truth')
plt.subplot(142)
plt.imshow(res['Uncorrected'][1][0,...,0])
plt.axis('off')
plt.title('Uncorrected')
plt.subplot(143)
plt.imshow(res['FB_0.0007'][1][0,...,0])
plt.axis('off')
plt.title('For.-Adj. Original TV param.')
plt.subplot(144)
plt.imshow(res['FB_0.01'][1][0,...,0])
plt.axis('off')
plt.title('For.-Adj. New TV param.')
# cbar_ax = fig.add_axes([0.93, 0.15, 0.015, 0.7])
# fig.colorbar(im, cax=cbar_ax)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
fig_name = 'TrainedVessels_EvalBalls.png'
vmax = image[0,...,0].max()
fig = plt.figure(figsize=(12,3))
plt.subplot(141)
im = plt.imshow(image[0,...,0]/vmax, vmin=0, vmax=1)
plt.axis('off')
plt.title('Ground Truth')
plt.subplot(142)
plt.imshow(res['AEM_0.0007'][1][0,...,0]/vmax, vmin=0, vmax=1)
plt.axis('off')
plt.title('AEM')
plt.subplot(143)
plt.imshow(res['FB_0.0007'][1][0,...,0]/vmax, vmin=0, vmax=1)
plt.axis('off')
plt.title('For.-Adj. Original TV param.')
plt.subplot(144)
plt.imshow(res['FB_0.01'][1][0,...,0]/vmax, vmin=0, vmax=1)
plt.axis('off')
plt.title('For.-Adj. New TV param.')
cbar_ax = fig.add_axes([0.93, 0.15, 0.015, 0.7])
fig.colorbar(im, cax=cbar_ax)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
fig_name = 'TrainedVessels_EvalBalls_renormalized.png'
# vmax = image[0,...,0].max()
fig = plt.figure(figsize=(12,3))
plt.subplot(141)
im = plt.imshow(image[0,...,0])
plt.axis('off')
plt.title('Ground Truth')
plt.subplot(142)
plt.imshow(res['AEM_0.0007'][1][0,...,0])
plt.axis('off')
plt.title('AEM')
plt.subplot(143)
plt.imshow(res['FB_0.0007'][1][0,...,0])
plt.axis('off')
plt.title('For.-Adj. Original TV param.')
plt.subplot(144)
plt.imshow(res['FB_0.01'][1][0,...,0])
plt.axis('off')
plt.title('For.-Adj. New TV param.')
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
n_fig = len(reg_param.keys())
for n_image in range(5):
    fig_name = f'Reconstuctions_{n_image}'
    fig = plt.figure(figsize=(15, 6))
    plt.subplot(1,n_fig+1,1)
    vmax=image[n_image,...,0].max()
    im = plt.imshow(image[n_image,...,0]/vmax, vmin=0, vmax=1)
#     plt.colorbar()
    plt.axis('off')
    plt.title('Ground Truth')
    for k, method in enumerate(reg_param):
        plt.subplot(1,n_fig+1,k+2)
        plt.imshow(res_aem[method][1][n_image,...,0]/vmax, vmin=0, vmax=1)
        plt.axis('off')
        plt.title(method)
    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.93, 0.15, 0.015, 0.7])
    fig.colorbar(im, cax=cbar_ax)
#     plt.colorbar()
#     plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
    plt.show()

In [ ]:
classical.end()
regularized.end()

# Experiment to obtain images for forward and gradient on ball data

In [ ]:
TV = 7e-4
regularized = TwoNets(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         experiment_name='TwoNetsRekursiveOldNet', savepoint=None)

In [ ]:
image = data_sets.test.default_batch(16)
x, data = regularized.sess.run([regularized.x_ini, regularized.measurement], feed_dict={regularized.input_image: image})
x = np.maximum(x, 0)

In [ ]:
for k in range(1):
    x, track = regularized.step(x, data, .001, 0.2)

In [ ]:
i = 0
vmin = min([track['TrueGrad'][i,...].min(), track['ApproxGrad'][i,...].min(), track['CorrectedGrad'][i,...].min()])
vmax = max([track['TrueGrad'][i,...].max(), track['ApproxGrad'][i,...].max(), track['CorrectedGrad'][i,...].max()])
print(vmin, vmax)
plt.figure(figsize=(10,10))
plt.subplot(131)
plt.imshow(track['TrueGrad'][i,...,0], vmin=vmin, vmax=vmax)
plt.title('True Gradient')
plt.axis('off')
plt.subplot(132)
plt.imshow(track['ApproxGrad'][i,...,0], vmin=vmin, vmax=vmax)
plt.title('Approximate Gradient')
plt.axis('off')
plt.subplot(133)
plt.imshow(track['CorrectedGrad'][i,...,0], vmin=vmin, vmax=vmax)
plt.title('Corrected Gradient')
plt.axis('off')
plt.savefig(save_dir+'Gradients_Iteration_0')
plt.show()

In [ ]:
vmin = min([track['TrueY'][i,...].min(), track['ApproxY'][i,...].min(), track['CorrectedY'][i,...].min()])
vmax = max([track['TrueY'][i,...].max(), track['ApproxY'][i,...].max(), track['CorrectedY'][i,...].max()])
print(vmin, vmax)
plt.figure(figsize=(10,10))
plt.subplot(131)
plt.imshow(track['TrueY'][i,...,0])
plt.title('True Measurement')
plt.axis('off')
plt.subplot(132)
plt.imshow(track['ApproxY'][i,...,0])
plt.title('Approximate Measurement')
plt.axis('off')
plt.subplot(133)
plt.imshow(track['CorrectedY'][i,...,0])
plt.title('Corrected Measurement')
plt.axis('off')
plt.savefig(save_dir+'Measurements_Iteration_0')
plt.show()

# Experiments to obtain model performance graphs

# Evaluations Vessel Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import random
import os
import platform
from Adjoint_regularizition import Regularized
from Adjoint_network import TwoNets
from Classical_correction import ClassicalCorrection
import Operators.Load_PAT2D_data as PATdata
import platform
from Framework import approx_PAT_matrix as ApproxPAT
from Framework import exact_PAT_operator as ExactPAT

if platform.node() == 'motel':
    prefix = '/local/scratch/public/sl767/ModelCorrection/'
else:
    prefix = ''

matrix_path = prefix+'Data/Matrices/threshSingleMatrix4Py.mat'

In [ ]:
data_path = prefix+'Data/vessels/'
saves_path = prefix+'Saves/vessels/'

print(saves_path)
print(data_path)

train_append = 'trainDataSet_VesselClean.mat'
test_append = 'testDataSet_VesselClean.mat'
data_sets = PATdata.read_data_sets(data_path + train_append, data_path + test_append, vessels=False, flip90=False)

In [ ]:
INPUT_DIM = (64,64)
OUTPUT_DIM = (64,64)

approx = ApproxPAT(matrix_path=matrix_path, input_dim=INPUT_DIM, output_dim=OUTPUT_DIM)
exact = ExactPAT(matrix_path=matrix_path, input_dim=INPUT_DIM, output_dim=OUTPUT_DIM)

In [ ]:
image = data_sets.test.default_batch(32)

In [ ]:
exp_name = 'TV'
exp_name_rec = 'TV_0.0007'
n_print = 25
n_steps = 250
verbose = False
noise_level = 0.01
TV = 7e-4
image = data_sets.test.default_batch(32)
res = {}

In [ ]:
exp_name = 'TV'
TV = 7e-4
regularized = Regularized(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         experiment_name='RegularizedAdjoint' + exp_name, savepoint=None)
res['Forward'] = regularized.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
regularized.end()

In [ ]:
exp_name_rec = 'TV_0.0007'
TV = 7e-4
regularized = Regularized(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         experiment_name='RegularizedAdjointRekursive' + exp_name_rec, savepoint=None)
res['Forward Rec.'] = regularized.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
regularized.end()

In [ ]:
exp_name = 'TV'
TV = 7e-4
twoNets = TwoNets(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         experiment_name='TwoNets' + exp_name, savepoint=None)
res['Forward-Adjoint'] = twoNets.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
twoNets.end()

In [ ]:
exp_name_rec = 'Aug'
TV = 3e-4
twoNets = TwoNets(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         experiment_name='TwoNetsRekursive' + exp_name_rec, savepoint=None)
res['Forward-Adjoint Rec.'] = twoNets.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
twoNets.end()

In [ ]:
model = ClassicalCorrection(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         noise_level = .01, experiment_name='ClassicalRegularization')

In [ ]:
TV = 3e-4
res['Accurate'] = model.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print, operator='True')

In [ ]:
TV = 7e-4
res['Uncorrected'] = model.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print, operator='Approx')

In [ ]:
### Best parameter: .00001
n_steps = 16000
n_print = 2000
TV = .00001
res['AEM'] = model.log_optimization(image, recursions=n_steps, step_size=0.5, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)

In [ ]:
model.end()

In [ ]:
fig_path = '/local/scratch/public/sl767/ModelCorrection/Figures/vessels/'

In [ ]:
#### ATTENTION! Executing this cell overwrites existing savepoints!!! #####
results_path = fig_path + 'Resluts/'
# os.mkdir(results_path)
for k, v in res.items():
    np.savez(results_path+k+'.npz', **(v[0]))
    np.savez(results_path+k+'_images.npz', *v[1])

In [ ]:
# Loading savepoint
results_path = fig_path + 'Resluts/'
res_loaded = {}
methods = ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.', 'Accurate', 'Uncorrected', 'AEM']
for m in methods:
    res_loaded[m] = []
    res_loaded[m].append(np.load(results_path+ m +'.npz'))
    res_loaded[m].append(np.load(results_path+ m +'_images.npz'))

# Generating figures - vessel data

In [ ]:
res.keys()

In [ ]:
color_dic = {
    'Forward': 'b',
    'Forward Rec.': 'm', 
    'Forward-Adjoint': 'g', 
    'Forward-Adjoint Rec.': 'c',
    'AEM': 'k',
    'AEM Conv.': 'k',
    'Accurate': 'k',
    'Uncorrected': 'r'
}

# b: blue
# g: green
# r: red
# c: cyan
# m: magenta
# y: yellow
# k: black
# w: white

line_types = {
    'Forward': '--',
    'Forward Rec.': '--', 
    'Forward-Adjoint': '-.', 
    'Forward-Adjoint Rec.': '-.',
    'AEM': ':',
    'AEM Conv.': '--',
    'Accurate': '-',
    'Uncorrected': '-'
}

In [ ]:
res['AEM Conv.'] = [{
    'quality': 250*[0.6511457562446594]
}]

n_steps = 250
# Plot Quality'
fig_name = 'quality'
methods= res.keys()
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['quality'][:n_steps], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(1.1, -0.12), ncol=3, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Relative Error (L2)', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

del res['AEM Conv.']

In [ ]:
# Plot Angle'
n_steps = 250
fig_name = 'angle'
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.', 'AEM']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['angle'][:n_steps], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(1., -0.12), ncol=3, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Alignement to true gradient', fontsize=14)
# plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
fig_name = 'ForwardError'
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['rel_loss_fwd'], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Relative error of forward approximation', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
fig_name = 'ForwardError_abs'
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['loss_fwd'], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Error of forward approximation (L2)', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
fig_name = 'AdjError'
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['rel_loss_adj'], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Relative error of adjoint approximation', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
fig_name = 'AdjError_abs'
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['loss_adj'], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Error of adjoint approximation (L2)', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Angle'
fig_name = 'dataTerm'
methods= res.keys()
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['DataTerm'][:n_steps], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(1.1, -0.12), ncol=3, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Data term for true operator', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
f_size = 18
for n_image in range(30):
    fig_name = f'Reconstuctions_{n_image}'
    fig = plt.figure(figsize=(15, 6))
    methods = ['Accurate', 'Uncorrected', 'AEM']
    plt.subplot(2,4,1)
    vmax=image[n_image,...,0].max()
    im = plt.imshow(image[n_image,...,0]/vmax, vmin=0, vmax=1)
#     plt.colorbar()
    plt.axis('off')
    plt.title('Ground Truth', fontsize=f_size)
    for k, method in enumerate(methods):
        plt.subplot(2,4,k+2)
        plt.imshow(res[method][1][f'arr_{n_image}'][...,0]/vmax, vmin=0, vmax=1)
        plt.axis('off')
        if method in ['True', 'Uncorrected']:
            method += ' Operator'
        plt.title(method, fontsize=f_size)
    methods = ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
    for k, method in enumerate(methods):
        plt.subplot(2,4,k+5)
        plt.imshow(res[method][1][f'arr_{n_image}'][...,0]/vmax, vmin=0, vmax=1)
        plt.axis('off')
        plt.title(method, fontsize=f_size)
    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.93, 0.15, 0.015, 0.7])
    fig.colorbar(im, cax=cbar_ax)
#     plt.colorbar()
    plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
    plt.show()

# Ball Data Evaluation

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import random
import os
from Adjoint_regularizition import Regularized
from Adjoint_network import TwoNets
from Classical_correction import ClassicalCorrection
import Operators.Load_PAT2D_data as PATdata
import platform
from Framework import approx_PAT_matrix as ApproxPAT
from Framework import exact_PAT_operator as ExactPAT

if platform.node() == 'motel':
    prefix = '/local/scratch/public/sl767/ModelCorrection/'
else:
    prefix = ''

matrix_path = prefix+'Data/Matrices/threshSingleMatrix4Py.mat'

In [ ]:
data_path = prefix+'Data/balls64/'
saves_path = prefix+'Saves/balls64/'

print(saves_path)
print(data_path)

train_append = 'trainDataSet.mat'
test_append = 'testDataSet.mat'
data_sets = PATdata.read_data_sets(data_path + train_append, data_path + test_append, vessels=False, flip90=False)

image= data_sets.test.default_batch(32)

In [ ]:
INPUT_DIM = (64,64)
OUTPUT_DIM = (64,64)

approx = ApproxPAT(matrix_path=matrix_path, input_dim=INPUT_DIM, output_dim=OUTPUT_DIM)
exact = ExactPAT(matrix_path=matrix_path, input_dim=INPUT_DIM, output_dim=OUTPUT_DIM)

In [ ]:
n_print = 100
n_steps = 4000
verbose = True
noise_level = 0.01
# TV = 7e-4
TV = 1e-3
image = data_sets.test.default_batch(32)
res = {}

In [ ]:
regularized = Regularized(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         experiment_name='RegularizedAdjointOldNet', savepoint=None)
res['Forward'] = regularized.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
regularized.end()

In [ ]:
regularized = Regularized(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         experiment_name='RegularizedAdjointRekursiveOldNet', savepoint=None)
res['Forward Rec.'] = regularized.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
regularized.end()

In [ ]:
twoNets = TwoNets(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         experiment_name='TwoNetsOldNet', savepoint=None)
res['Forward-Adjoint'] = twoNets.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
twoNets.end()

In [ ]:
twoNets = TwoNets(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         experiment_name='TwoNetsRekursiveOldNet', savepoint=None)
res['Forward-Adjoint Rec.'] = twoNets.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)
twoNets.end()

In [ ]:
model = ClassicalCorrection(path=saves_path, true_np=exact, appr_np=approx, lam=TV, data_sets=data_sets,
                         noise_level = .01, experiment_name='ClassicalRegularization')

In [ ]:
res['Accurate'] = model.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print, operator='True')

In [ ]:
res['Uncorrected'] = model.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print, operator='Approx')

In [ ]:
res['AEM'] = model.log_optimization(image, recursions=n_steps, step_size=0.2, lam=TV, 
                       verbose=verbose, tensorboard=False, n_print=n_print)

In [ ]:
model.end()

In [ ]:
fig_path = '/local/scratch/public/sl767/ModelCorrection/Figures/balls/'

In [ ]:
#### ATTENTION! Executing this cell overwrites existing savepoints!!! #####
results_path = fig_path + 'Resluts/'
# os.mkdir(results_path)
for k, v in res.items():
    np.savez(results_path+k+'.npz', **(v[0]))
    np.savez(results_path+k+'_images.npz', *v[1])

In [ ]:
results_path = fig_path + 'Resluts/'
res_loaded = {}
methods = ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.', 'Accurate', 'Uncorrected', 'AEM']
for m in methods:
    res_loaded[m] = []
    res_loaded[m].append(np.load(results_path+ m +'.npz'))
    res_loaded[m].append(np.load(results_path+ m +'_images.npz'))

In [ ]:
res = res_loaded

# Generating figures - ball data

In [ ]:
res.keys()

In [ ]:
color_dic = {
    'Forward': 'b',
    'Forward Rec.': 'm', 
    'Forward-Adjoint': 'g', 
    'Forward-Adjoint Rec.': 'c',
    'AEM': 'k',
    'AEM Conv.': 'k',
    'Accurate': 'k',
    'Uncorrected': 'r'
}

# b: blue
# g: green
# r: red
# c: cyan
# m: magenta
# y: yellow
# k: black
# w: white

line_types = {
    'Forward': '--',
    'Forward Rec.': '--', 
    'Forward-Adjoint': '-.', 
    'Forward-Adjoint Rec.': '-.',
    'AEM': ':',
    'AEM Conv.': '--',
    'Accurate': '-',
    'Uncorrected': '-'
}

In [ ]:
n_steps = 4000
# Plot Quality'
fig_name = 'quality'
methods= res.keys()
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['quality'][:n_steps], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(1.1, -0.12), ncol=3, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Relative Error (L2)', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

# del res['AEM Conv.']

In [ ]:
# Plot Angle'
n_steps = 4000
fig_name = 'angle'
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.', 'AEM']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps)[0::50], res[k][0]['angle'][:n_steps][0::50], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(1., -0.12), ncol=3, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Alignement to true gradient', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Angle
fig_name = 'angle_500'
n_steps = 500
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.', 'AEM']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps)[0::10], res[k][0]['angle'][:n_steps][0::10], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(1., -0.12), ncol=3, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Alignement to true gradient', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
fig_name = 'ForwardError'
n_steps=4000
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['rel_loss_fwd'], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Relative error of forward approximation', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
fig_name = 'ForwardError_abs'
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['loss_fwd'], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Error of forward approximation (L2)', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
fig_name = 'ForwardError_300'
n_steps=300
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['rel_loss_fwd'][:n_steps], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Relative Error of forward approximation', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
fig_name = 'ForwardError_abs_300'
n_steps=300
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['loss_fwd'][:n_steps], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Error of forward approximation (L2)', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
n_steps = 4000
fig_name = 'AdjError'
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['rel_loss_adj'], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Relative error of adjoint approximation', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Forward Approximation'
fig_name = 'AdjError_abs'
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['loss_adj'], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Error of adjoint approximation (L2)', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
fig_name = 'AdjError_300'
n_steps=300
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['rel_loss_adj'][:n_steps], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Relative error of adjoint approximation', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
fig_name = 'AdjError_abs_300'
n_steps = 300
methods= ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['loss_adj'][:n_steps], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(.9, -0.12), ncol=2, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Error of adjoint approximation (L2)', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
# Plot Angle'
fig_name = 'dataTerm'
methods= res.keys()
plt.figure(figsize=(7,7))
for k in methods:
    plt.plot(range(n_steps), res[k][0]['DataTerm'][:n_steps], color_dic[k]+line_types[k], label=k)
plt.legend(bbox_to_anchor=(1.1, -0.12), ncol=3, fontsize=12)
plt.xlabel('Optimisation Steps', fontsize=14)
plt.ylabel('Data term for true operator', fontsize=14)
plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
f_size = 18
for n_image in range(30):
    fig_name = f'Reconstuctions_{n_image}'
    fig = plt.figure(figsize=(15, 6))
    methods = ['Accurate', 'Uncorrected', 'AEM']
    plt.subplot(2,4,1)
    vmax=image[n_image,...,0].max()
    im = plt.imshow(image[n_image,...,0]/vmax, vmin=0, vmax=1)
#     plt.colorbar()
    plt.axis('off')
    plt.title('Ground Truth', fontsize=f_size)
    for k, method in enumerate(methods):
        plt.subplot(2,4,k+2)
        plt.imshow(res[method][1][f'arr_{n_image}'][...,0]/vmax, vmin=0, vmax=1)
        plt.axis('off')
        if method in ['True', 'Uncorrected']:
            method += ' Operator'
        plt.title(method, fontsize=f_size)
    methods = ['Forward', 'Forward Rec.', 'Forward-Adjoint', 'Forward-Adjoint Rec.']
    for k, method in enumerate(methods):
        plt.subplot(2,4,k+5)
        plt.imshow(res[method][1][f'arr_{n_image}'][...,0]/vmax, vmin=0, vmax=1)
        plt.axis('off')
        plt.title(method, fontsize=f_size)
    fig.subplots_adjust(right=0.9)
    cbar_ax = fig.add_axes([0.93, 0.15, 0.015, 0.7])
    fig.colorbar(im, cax=cbar_ax)
#     plt.colorbar()
    plt.savefig(fig_path+fig_name, dpi=300, bbox_inches='tight', transparent=True)
    plt.show()